In [23]:
! python -m pip install chembl_webresource_client
! python -m pip install pandas


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [25]:
# find target protein for acetylcholinesterase

target = new_client.target
target_query = target.search('acetylcholinesterase')
targets = pd.DataFrame.from_dict(target_query)
print(targets)

   cross_references                   organism  \
0                []    Drosophila melanogaster   
1                []               Homo sapiens   
2                []        Torpedo californica   
3                []               Mus musculus   
4                []          Rattus norvegicus   
5                []   Electrophorus electricus   
6                []                 Bos taurus   
7                []             Bemisia tabaci   
8                []  Leptinotarsa decemlineata   
9                []     Nephotettix cincticeps   
10               []              Gallus gallus   
11               []                Danio rerio   
12               []            Musca domestica   
13               []          Anopheles gambiae   
14               []               Homo sapiens   
15               []        Plutella xylostella   
16               []            Musca domestica   
17               []  Pediculus humanus capitis   
18               []          Spodoptera litura   


In [26]:
# selecting the first target protein
selected_target = targets.target_chembl_id[1] # selcting human as target orgnanism

# Here, we will retrieve only bioactivity data for Human Acetylcholinesterase (CHEMBL220) that are reported as pChEMBL values.
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

df = pd.DataFrame.from_dict(res)

df


,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,12438748,[],CHEMBL3068296,Inhibition of acetylcholinesterase in Drosophi...,B,None,None,BAO_0000190,...,Drosophila melanogaster,Acetylcholinesterase,7227,None,None,IC50,uM,UO_0000065,None,1.83
1,None,None,12438749,[],CHEMBL3068296,Inhibition of acetylcholinesterase in Drosophi...,B,None,None,BAO_0000190,...,Drosophila melanogaster,Acetylcholinesterase,7227,None,None,IC50,uM,UO_0000065,None,112.72
2,None,None,12438750,[],CHEMBL3068296,Inhibition of acetylcholinesterase in Drosophi...,B,None,None,BAO_0000190,...,Drosophila melanogaster,Acetylcholinesterase,7227,None,None,IC50,uM,UO_0000065,None,37.5
3,None,None,12438751,[],CHEMBL3068296,Inhibition of acetylcholinesterase in Drosophi...,B,None,None,BAO_0000190,...,Drosophila melanogaster,Acetylcholinesterase,7227,None,None,IC50,uM,UO_0000065,None,4.5
4,None,None,12438752,[],CHEMBL3068296,Inhibition of acetylcholinesterase in Drosophi...,B,None,None,BAO_0000190,...,Drosophila melanogaster,Acetylcholinesterase,7227,None,None,IC50,uM,UO_0000065,None,1.84
5,None,None,12438753,[],CHEMBL3068296,Inhibition of acetylcholinesterase in Drosophi...,B,None,None,BAO_0000190,...,Drosophila melanogaster,Acetylcholinesterase,7227,None,None,IC50,uM,UO_0000065,None,3.81
6,None,None,12438754,[],CHEMBL3068296,Inhibition of acetylcholinesterase in Drosophi...,B,None,None,BAO_0000190,...,Drosophila melanogaster,Acetylcholinesterase,7227,None,None,IC50,uM,UO_0000065,None,12.34
7,None,None,12438755,[],CHEMBL3068296,Inhibition of acetylcholinesterase in Drosophi...,B,None,None,BAO_0000190,...,Drosophila melanogaster,Acetylcholinesterase,7227,None,None,IC50,uM,UO_0000065,None,9.92
8,None,None,12438756,[],CHEMBL3068296,Inhibition of acetylcholinesterase in Drosophi...,B,None,None,BAO_0000190,...,Drosophila melanogaster,Acetylcholinesterase,7227,None,None,IC50,uM,UO_0000065,None,8.29
9,None,None,12438757,[],CHEMBL3068296,Inhibition of acetylcholinesterase in Drosophi...,B,None,None,BAO_0000190,...,Drosophila melanogaster,Acetylcholinesterase,7227,None,None,IC50,uM,UO_0000065,None,1.76


In [27]:
df.to_csv('acetylcholinesterase_01_bioactivity_data_raw.csv', index=False)

Handling missing data

In [28]:
df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]

In [29]:
# drop duplicates, smiles is for chemical compound
df2_nr = df2.drop_duplicates(['canonical_smiles'])

In [30]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_nr[selection]

In [31]:
df3.to_csv('acetylcholinesterase_02_bioactivity_data_preprocessed.csv', index=False)

Labelling compund as active/inactive

In [32]:
df4 = pd.read_csv('acetylcholinesterase_02_bioactivity_data_preprocessed.csv')


bioactivity_threshold = []
for i in df4.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1000:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")


bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)
df5

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL463210,CCOP(=S)(OCC)Oc1nc(Cl)c(Cl)cc1Cl,1830.0,intermediate
1,CHEMBL2252723,CCOP(=O)(OCC)SCCCCCCCCCCN1C(=O)c2ccccc2C1=O,112720.0,inactive
2,CHEMBL2252722,CCOP(=O)(OCC)SCCCCCCCCCN1C(=O)c2ccccc2C1=O,37500.0,inactive
3,CHEMBL2252721,CCOP(=O)(OCC)SCCCCCCCCN1C(=O)c2ccccc2C1=O,4500.0,intermediate
4,CHEMBL2252851,CCOP(=O)(OCC)SCCCCCCCN1C(=O)c2ccccc2C1=O,1840.0,intermediate
5,CHEMBL2252850,CCOP(=O)(OCC)SCCCCCCN1C(=O)c2ccccc2C1=O,3810.0,intermediate
6,CHEMBL2252849,CCOP(=O)(OCC)SCCCCCN1C(=O)c2ccccc2C1=O,12340.0,inactive
7,CHEMBL2252848,CCOP(=O)(OCC)SCCCCN1C(=O)c2ccccc2C1=O,9920.0,intermediate
8,CHEMBL2252847,CCOP(=O)(OCC)SCCCN1C(=O)c2ccccc2C1=O,8290.0,intermediate
9,CHEMBL2252846,CCOP(=O)(OCC)SCCCCCCCCCCSP(=O)(OCC)OCC,1760.0,intermediate


In [33]:
df5.to_csv('acetylcholinesterase_03_bioactivity_data_curated.csv', index=False)


! zip acetylcholinesterase.zip *.csv

'zip' is not recognized as an internal or external command,
operable program or batch file.
